In [ ]:
import cv2
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.preprocessing import image
from skimage.transform import resize
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, InputLayer

In [ ]:
count = 0
videoFile = "tomnjerry.mp4"
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="test%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")

Done!


In [ ]:


def capture_frames(video_file, output_prefix):
    cap = cv2.VideoCapture(video_file)
    frameRate = cap.get(5)  # Frame rate
    count = 0

    while cap.isOpened():
        frameId = cap.get(1)  # Current frame number
        ret, frame = cap.read()
        if not ret:
            break

        if frameId % math.floor(frameRate) == 0:
            filename = f"{output_prefix}{count}.jpg"
            cv2.imwrite(filename, frame)
            count += 1

    cap.release()

# Load an image and display it
def load_and_display_image(image_path):
    img = plt.imread(image_path)
    plt.imshow(img)
    plt.show()

# Load and preprocess the images
def load_and_preprocess_images(image_paths):
    images = []
    for img_path in image_paths:
        img = plt.imread(img_path)
        img = resize(img, preserve_range=True, output_shape=(224, 224)).astype(int)
        images.append(img)
    return np.array(images)

# Main function

video_file = "tomnjerry.mp4"
output_prefix = "frame"
mapping_csv = "mapping_29.csv"
testing_csv = "testing_29.csv"

# Step 1: Capture frames from the video
capture_frames(video_file, output_prefix)
print("Frames captured.")

# Step 2: Load and preprocess frames
data = pd.read_csv(mapping_csv)
X = load_and_preprocess_images(data.Image_id)
y = data.Class
dummy_y = to_categorical(y)

# Step 3: Load and preprocess test frames
test_data = pd.read_csv(testing_csv)
test_images = load_and_preprocess_images(test_data.Image_id)

# Step 4: Load VGG16 model as a feature extractor
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Extract features from frames using the VGG16 model
X_features = base_model.predict(X)
test_features = base_model.predict(test_images)

# Reshape the extracted features
X_features = X_features.reshape(X_features.shape[0], -1)
test_features = test_features.reshape(test_features.shape[0], -1)

# Step 5: Build and train a classification model
model = Sequential()
model.add(InputLayer(input_shape=(X_features.shape[1],)))
model.add(Dense(units=1024, activation='sigmoid'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history=model.fit(X_features, dummy_y, epochs=12)

# Step 6: Make predictions on test frames
predictions = model.predict(test_features)

# Calculate screen time
jerry_screen_time = predictions[:, 1].sum()  # Assuming class 1 corresponds to JERRY
tom_screen_time = predictions[:, 2].sum()    # Assuming class 2 corresponds to TOM

print("The screen time of JERRY is", jerry_screen_time, "seconds")
print("The screen time of TOM is", tom_screen_time, "seconds")



Frames captured.
2/2 [==============================] - 13s 1s/step
Epoch 1/12
2/2 [==============================] - 1s 203ms/step - loss: 1.7098 - accuracy: 0.3143
Epoch 2/12
2/2 [==============================] - 0s 193ms/step - loss: 1.5221 - accuracy: 0.6857
Epoch 3/12
2/2 [==============================] - 0s 191ms/step - loss: 0.4626 - accuracy: 0.8286
Epoch 4/12
2/2 [==============================] - 0s 195ms/step - loss: 0.2165 - accuracy: 0.9143
Epoch 5/12
2/2 [==============================] - 0s 203ms/step - loss: 0.1761 - accuracy: 0.9143
Epoch 6/12
2/2 [==============================] - 0s 200ms/step - loss: 0.1558 - accuracy: 0.8857
Epoch 7/12
2/2 [==============================] - 0s 196ms/step - loss: 0.1371 - accuracy: 0.9143
Epoch 8/12
2/2 [==============================] - 0s 201ms/step - loss: 0.1287 - accuracy: 0.9429
Epoch 9/12
2/2 [==============================] - 0s 200ms/step - loss: 0.1122 - accuracy: 0.9714
Epoch 10/12
2/2 [==============================] -

In [ ]:
import cv2
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.preprocessing import image
from skimage.transform import resize
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

count = 0

def capture_frames(video_file, output_prefix):
    cap = cv2.VideoCapture(video_file)
    frameRate = cap.get(5)  # Frame rate
    count = 0

    while cap.isOpened():
        frameId = cap.get(1)  # Current frame number
        ret, frame = cap.read()
        if not ret:
            break

        if frameId % math.floor(frameRate) == 0:
            filename = f"{output_prefix}{count}.jpg"
            cv2.imwrite(filename, frame)
            count += 1

    cap.release()

# Load and preprocess the images
def load_and_preprocess_images(image_paths):
    images = []
    for img_path in image_paths:
        img = plt.imread(img_path)
        img = resize(img, preserve_range=True, output_shape=(224, 224)).astype(int)
        images.append(img)
    return np.array(images)

# Main function
video_file = "tomnjerry.mp4"
output_prefix = "frame"
mapping_csv = "mapping_29.csv"
testing_csv = "testing_29.csv"

# Step 1: Capture frames from the video
capture_frames(video_file, output_prefix)
print("Frames captured.")

# Step 2: Load and preprocess frames
data = pd.read_csv(mapping_csv)
X = load_and_preprocess_images(data.Image_id)
y = data.Class
dummy_y = to_categorical(y)

# Step 3: Load and preprocess test frames
test_data = pd.read_csv(testing_csv)
test_images = load_and_preprocess_images(test_data.Image_id)

# Step 4: Build and train a CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
history = model.fit(X, dummy_y, epochs=10)

# Step 5: Make predictions on test frames
predictions = model.predict(test_images)

# Calculate screen time
jerry_screen_time = predictions[:, 1].sum()  # Assuming class 1 corresponds to JERRY
tom_screen_time = predictions[:, 2].sum()    # Assuming class 2 corresponds to TOM

print("The screen time of JERRY is", jerry_screen_time, "seconds")
print("The screen time of TOM is", tom_screen_time, "seconds")


Frames captured.
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 54, 54, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_11 (Conv2D)          (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 2

In [ ]:
import cv2
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from skimage.transform import resize
from keras.utils import to_categorical
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam

count = 0

video_file = "tomnjerry.mp4"
output_prefix = "frame"
mapping_csv = "mapping_29.csv"
testing_csv = "testing_29.csv"

def capture_frames(video_file, output_prefix):
    cap = cv2.VideoCapture(video_file)
    frameRate = cap.get(5)  # Frame rate
    count = 0

    while cap.isOpened():
        frameId = cap.get(1)  # Current frame number
        ret, frame = cap.read()
        if not ret:
            break

        if frameId % math.floor(frameRate) == 0:
            filename = f"{output_prefix}{count}.jpg"
            cv2.imwrite(filename, frame)
            count += 1

    cap.release()

def load_and_preprocess_images(image_paths):
    images = []
    for img_path in image_paths:
        img = plt.imread(img_path)
        img = resize(img, preserve_range=True, output_shape=(224, 224)).astype(int)
        images.append(img)
    return np.array(images)

# Step 1: Capture frames from the video
capture_frames(video_file, output_prefix)
print("Frames captured.")

# Step 2: Load and preprocess frames
data = pd.read_csv(mapping_csv)
X = load_and_preprocess_images(data.Image_id)
y = data.Class
dummy_y = to_categorical(y)

# Step 3: Load and preprocess test frames
test_data = pd.read_csv(testing_csv)
test_images = load_and_preprocess_images(test_data.Image_id)

# Step 4: Load ResNet-50 model as a feature extractor
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Extract features from frames using the ResNet-50 model
X_features = base_model.predict(preprocess_input(X))
test_features = base_model.predict(preprocess_input(test_images))

# Step 5: Build and train a classification model
model = Sequential()
model.add(GlobalAveragePooling2D(input_shape=X_features.shape[1:]))
model.add(Dense(1024, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

history = model.fit(X_features, dummy_y, epochs=10)

# Step 6: Make predictions on test frames
predictions = model.predict(test_features)

# Calculate screen time
jerry_screen_time = predictions[:, 1].sum()  # Assuming class 1 corresponds to JERRY
tom_screen_time = predictions[:, 2].sum()    # Assuming class 2 corresponds to TOM

print("The screen time of JERRY is", jerry_screen_time, "seconds")
print("The screen time of TOM is", tom_screen_time, "seconds")


Frames captured.
2/2 [==============================] - 4s 310ms/step


Epoch 1/10
2/2 [==============================] - 0s 16ms/step - loss: 2.7404 - accuracy: 0.3143
Epoch 2/10
2/2 [==============================] - 0s 15ms/step - loss: 6.4007 - accuracy: 0.5143
Epoch 3/10
2/2 [==============================] - 0s 17ms/step - loss: 0.4025 - accuracy: 0.8286
Epoch 4/10
2/2 [==============================] - 0s 13ms/step - loss: 2.4730 - accuracy: 0.5429
Epoch 5/10
2/2 [==============================] - 0s 15ms/step - loss: 1.5202 - accuracy: 0.6857
Epoch 6/10
2/2 [==============================] - 0s 14ms/step - loss: 0.3046 - accuracy: 0.8571
Epoch 7/10
2/2 [==============================] - 0s 13ms/step - loss: 0.2423 - accuracy: 0.9429
Epoch 8/10
2/2 [==============================] - 0s 13ms/step - loss: 0.1546 - accuracy: 0.9714
Epoch 9/10
2/2 [==============================] - 0s 12ms/step - loss: 0.1172 - accuracy: 0.9429
Epoch 10/10
2/2 [==============================] - 0s 5ms/step
The screen time of JERRY is 10.180044 seconds
The screen time of

In [ ]:
import cv2
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from skimage.transform import resize
from keras.utils import to_categorical
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam

count = 0

videoFile = "tomnjerry.mp4"
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5)  # frame rate
x = 1
while cap.isOpened():
    frameId = cap.get(1)  # current frame number
    ret, frame = cap.read()
    if not ret:
        break
    if frameId % math.floor(frameRate) == 0:
        filename = "test%d.jpg" % count
        count += 1
        cv2.imwrite(filename, frame)
cap.release()
print("Done!")

def capture_frames(video_file, output_prefix):
    cap = cv2.VideoCapture(video_file)
    frameRate = cap.get(5)  # Frame rate
    count = 0

    while cap.isOpened():
        frameId = cap.get(1)  # Current frame number
        ret, frame = cap.read()
        if not ret:
            break

        if frameId % math.floor(frameRate) == 0:
            filename = f"{output_prefix}{count}.jpg"
            cv2.imwrite(filename, frame)
            count += 1

    cap.release()

# Load an image and display it
def load_and_display_image(image_path):
    img = plt.imread(image_path)
    plt.imshow(img)
    plt.show()

# Load and preprocess the images
def load_and_preprocess_images(image_paths):
    images = []
    for img_path in image_paths:
        img = plt.imread(img_path)
        img = resize(img, preserve_range=True, output_shape=(299, 299)).astype(int)
        images.append(img)
    return np.array(images)

# Main function
video_file = "tomnjerry.mp4"
output_prefix = "frame"
mapping_csv = "mapping_29.csv"
testing_csv = "testing_29.csv"

# Step 1: Capture frames from the video
capture_frames(video_file, output_prefix)
print("Frames captured.")

# Step 2: Load and preprocess frames
data = pd.read_csv(mapping_csv)
X = load_and_preprocess_images(data.Image_id)
y = data.Class
dummy_y = to_categorical(y)

# Step 3: Load and preprocess test frames
test_data = pd.read_csv(testing_csv)
test_images = load_and_preprocess_images(test_data.Image_id)

# Step 4: Load InceptionV3 model as a feature extractor
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Extract features from frames using the InceptionV3 model
X_features = base_model.predict(preprocess_input(X))
test_features = base_model.predict(preprocess_input(test_images))

# Step 5: Build and train a classification model
model = Sequential()
model.add(GlobalAveragePooling2D(input_shape=X_features.shape[1:]))
model.add(Dense(1024, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

history = model.fit(X_features, dummy_y, epochs=10)

# Step 6: Make predictions on test frames
predictions = model.predict(test_features)

# Calculate screen time
jerry_screen_time = predictions[:, 1]  # Assuming class 1 corresponds to JERRY
tom_screen_time = predictions[:, 2]    # Assuming class 2 corresponds to TOM

print("The screen time of JERRY is", jerry_screen_time.sum(), "seconds")
print("The screen time of TOM is", tom_screen_time.sum(), "seconds")


Done!
Frames captured.
2/2 [==============================] - 5s 574ms/step


Epoch 1/10
2/2 [==============================] - 0s 14ms/step - loss: 1.4013 - accuracy: 0.3429
Epoch 2/10
2/2 [==============================] - 0s 14ms/step - loss: 3.9967 - accuracy: 0.3714
Epoch 3/10
2/2 [==============================] - 0s 16ms/step - loss: 3.0454 - accuracy: 0.5429
Epoch 4/10
2/2 [==============================] - 0s 15ms/step - loss: 0.3826 - accuracy: 0.8286
Epoch 5/10
2/2 [==============================] - 0s 18ms/step - loss: 0.6323 - accuracy: 0.6571
Epoch 6/10
2/2 [==============================] - 0s 15ms/step - loss: 0.9466 - accuracy: 0.6000
Epoch 7/10
2/2 [==============================] - 0s 14ms/step - loss: 0.3526 - accuracy: 0.8286
Epoch 8/10
2/2 [==============================] - 0s 14ms/step - loss: 0.3856 - accuracy: 0.8571
Epoch 9/10
2/2 [==============================] - 0s 12ms/step - loss: 0.2452 - accuracy: 0.9714
Epoch 10/10
2/2 [==============================] - 0s 6ms/step
The screen time of JERRY is 13.872986 seconds
The screen time of

In [ ]:
import cv2
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from skimage.transform import resize
from keras.utils import to_categorical
from keras.applications import DenseNet121
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam

count = 0

videoFile = "tomnjerry.mp4"
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5)  # frame rate
x = 1
while cap.isOpened():
    frameId = cap.get(1)  # current frame number
    ret, frame = cap.read()
    if not ret:
        break
    if frameId % math.floor(frameRate) == 0:
        filename = "test%d.jpg" % count
        count += 1
        cv2.imwrite(filename, frame)
cap.release()
print("Done!")

def capture_frames(video_file, output_prefix):
    cap = cv2.VideoCapture(video_file)
    frameRate = cap.get(5)  # Frame rate
    count = 0

    while cap.isOpened():
        frameId = cap.get(1)  # Current frame number
        ret, frame = cap.read()
        if not ret:
            break

        if frameId % math.floor(frameRate) == 0:
            filename = f"{output_prefix}{count}.jpg"
            cv2.imwrite(filename, frame)
            count += 1

    cap.release()

# Load and preprocess the images
def load_and_preprocess_images(image_paths):
    images = []
    for img_path in image_paths:
        img = plt.imread(img_path)
        img = resize(img, preserve_range=True, output_shape=(224, 224)).astype(int)
        images.append(img)
    return np.array(images)

# Main function
video_file = "tomnjerry.mp4"
output_prefix = "frame"
mapping_csv = "mapping_29.csv"
testing_csv = "testing_29.csv"

# Step 1: Capture frames from the video
capture_frames(video_file, output_prefix)
print("Frames captured.")

# Step 2: Load and preprocess frames
data = pd.read_csv(mapping_csv)
X = load_and_preprocess_images(data.Image_id)
y = data.Class
dummy_y = to_categorical(y)

# Step 3: Load and preprocess test frames
test_data = pd.read_csv(testing_csv)
test_images = load_and_preprocess_images(test_data.Image_id)

# Step 4: Load DenseNet model as a feature extractor
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Extract features from frames using the DenseNet model
X_features = base_model.predict(X)
test_features = base_model.predict(test_images)

# Step 5: Build and train a classification model
model = Sequential()
model.add(GlobalAveragePooling2D(input_shape=X_features.shape[1:]))
model.add(Dense(1024, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

history = model.fit(X_features, dummy_y, epochs=10)

# Step 6: Make predictions on test frames
predictions = model.predict(test_features)

# Calculate screen time
jerry_screen_time = predictions[:, 1]  # Assuming class 1 corresponds to JERRY
tom_screen_time = predictions[:, 2]    # Assuming class 2 corresponds to TOM

print("The screen time of JERRY is", jerry_screen_time.sum(), "seconds")
print("The screen time of TOM is", tom_screen_time.sum(), "seconds")


Done!
Frames captured.
2/2 [==============================] - 3s 367ms/step


Epoch 1/10
2/2 [==============================] - 0s 9ms/step - loss: 11.2843 - accuracy: 0.4000
Epoch 2/10
2/2 [==============================] - 0s 11ms/step - loss: 26.1278 - accuracy: 0.4286
Epoch 3/10
2/2 [==============================] - 0s 10ms/step - loss: 19.2875 - accuracy: 0.3714
Epoch 4/10
2/2 [==============================] - 0s 10ms/step - loss: 6.1248 - accuracy: 0.6571
Epoch 5/10
2/2 [==============================] - 0s 13ms/step - loss: 10.6008 - accuracy: 0.6000
Epoch 6/10
2/2 [==============================] - 0s 10ms/step - loss: 6.6810 - accuracy: 0.6000
Epoch 7/10
2/2 [==============================] - 0s 11ms/step - loss: 0.6567 - accuracy: 0.8286
Epoch 8/10
2/2 [==============================] - 0s 10ms/step - loss: 2.0393 - accuracy: 0.7143
Epoch 9/10
2/2 [==============================] - 0s 10ms/step - loss: 1.1933 - accuracy: 0.8286
Epoch 10/10
2/2 [==============================] - 0s 5ms/step
The screen time of JERRY is 8.051988 seconds
The screen time 